# Tailscale Installation Script Security Audits

The corresponding notebook/post includes directions to download and run installation scripts, but I'm always a little wary of running shell scripts I haven't looked over, even if they're from a reliable-seeming source. So I'll scan over install scripts

### Tailscale Client Installation on a Ubuntu/Debian Linux Machine

```console
curl -fsSL https://tailscale.com/install.sh | sh
```

In [1]:
#| code-fold: fold
#| code-summary: imports

from IPython.display import display, HTML
import requests

In [2]:
#| code-fold: show
#| code-summary: Getting the install script

url = "https://tailscale.com/install.sh"
resp = requests.get(url)
resp.raise_for_status()
linux_script_lines = resp.text
lines_in_script = len(linux_script_lines.split('\n'))
print(f"Lines in install script: {lines_in_script}")

Lines in install script: 627


The install script defines and invokes one function, `main()`.

The `main()` function starts off by defining variables for the invoking-machine's:
* OS,
* OS version,
* OS package manager type,
* package signing key type (for distros using the `apt` package manager, e.g. Debian/Ubuntu; options: "legacy" or "keyring"),
* a boolean flag so that `systemctl` commands can be run to enable tailscale to start on boot (only necessary on the Kali linux distro), and
* a `TRACK` variable that only accepts values "stable" or "unstable" (it defaults to "stable" if there's not already a `TRACK` env-var set, so I assume this is just to make it more ergonomic for devs to install nightly releases).

The first four are assigned empty-string placeholder values.

In [3]:
#| code-fold: show
#| code-summary: The install script's preamble

print("\n".join(linux_script_lines.split("\n")[12:36]))

main() {
	# Step 1: detect the current linux distro, version, and packaging system.
	#
	# We rely on a combination of 'uname' and /etc/os-release to find
	# an OS name and version, and from there work out what
	# installation method we should be using.
	#
	# The end result of this step is that the following three
	# variables are populated, if detection was successful.
	OS=""
	VERSION=""
	PACKAGETYPE=""
	APT_KEY_TYPE="" # Only for apt-based distros
	APT_SYSTEMCTL_START=false # Only needs to be true for Kali
	TRACK="${TRACK:-stable}"

	case "$TRACK" in
		stable|unstable)
			;;
		*)
			echo "unsupported track $TRACK"
			exit 1
			;;
	esac


The next block of code checks the invoking-machine for an `/etc/os-release` file, and if present, it "runs" the file, adding all of the file's variables to the shell's present context and updates the above placeholder variables based on the OS indicated by the file's `ID` variable.

If no `/etc/os-release is found, this block is skipped.

$$
f_{\text{OS}}(\text{ID}) =
\begin{cases}
\text{Ubuntu, if ID} \in \{\text{ubuntu, pop, neon, zorin, tuxedo, elementary, galliumos} \} \\
\text{Ubuntu, if ID} = \text{linuxmint } \& \text{ } [\text{UBUNTU\_CODENAME} \neq \text{""} \lor \text{ DEBIAN\_CODENAME} = \text{""}] \\
\text{Ubuntu, if ID} = \text{pika } \& \text{ VERSION\_ID} \lt 4 \\
\text{Debian, if ID} \in \{\text{debian, parrot, mendel, pureos, kaisen, kali, Deepin, deepin, osmc} \} \\
\text{Debian, if ID} = \text{linuxmint } \& \text{ DEBIAN\_CODENAME} \neq \text{""} \\
\text{Debian, if ID} = \text{pika } \& \text{ VERSION\_ID} \geq 4 \\
\text{Raspbian, if ID} \in \{\text{raspbian} \} \\
\text{CentOS, if ID} \in \{\text{centos} \} \\
\text{Oracle, if ID} \in \{\text{ol} \} \\
\text{Red Hat, if ID} \in \{\text{rhel} \} \\
\text{Fedora, if ID} \in \{\text{fedora, rocky, almalinux, nobara, openmandriva, sangoma, risios, cloudlinux, alinux, fedora-asahi-remix} \} \\
\text{Amazon-Linux, if ID} \in \{\text{amzn} \} \\
\text{CentOS, if ID} \in \{\text{xenenterprise} \} \\
\text{OpenSUSE, if ID} \in \{\text{opensuse-leap, sles, opensuse-tumbleweed, sle-micro-rancher} \} \\
\text{Arch, if ID} \in \{\text{arch, archarm, endeavouros, blendos, garuda, archcraft, cachyos} \} \\
\text{Manjaro, if ID} \in \{\text{manjaro, manjaro-arm} \} \\
\text{Alpine, if ID} \in \{\text{alpine, postmarketos} \} \\
\text{Void, if ID} \in \{\text{void} \} \\
\text{Gentoo, if ID} \in \{\text{gentoo} \} \\
\text{FreeBSD, if ID} \in \{\text{freebsd} \} \\
\text{Photon, if ID} \in \{\text{photon} \} \\
\end{cases}
$$

If $\text{ID} = \text{nixos}$, it prints a message directing the user to manually add Tailscale to their `NixOS` config and terminates the script early.

$$
f_{\text{PACKAGETYPE}}(\text{OS, ID}) =
\begin{cases}
\text{apt, if OS} \in \{ \text{Ubuntu, Debian} \} \\
\text{dnf, if OS} \in \{ \text{Fedora} \} \\
\text{dnf, if OS} \in \{ \text{CentOS, Oracle, Red Hat} \} \text{ } \& \text{ } [\lfloor \text{VERSION\_ID} \rfloor \neq 7] \text{ } \& \text{ ID} \neq \text{xenenterprise} \\
\text{yum, if OS} \in \{ \text{CentOS, Oracle, Red Hat} \} \text{ } \& \lfloor \text{VERSION\_ID} \rfloor = 7] \\
\text{yum, if OS} \in \{ \text{Amazon-Linux} \} \lor \text{ID} = \text{xenenterprise} \\
\text{yum, if OS} \in \{ \text{OpenSUSE} \} \\
\text{pacman, if OS} \in \{ \text{Arch, Manjaro} \} \\
\text{apk, if OS} \in \{ \text{Alpine} \} \\
\text{xbps, if OS} \in \{ \text{Void} \} \\
\text{emerge, if OS} \in \{ \text{Gentoo} \} \\
\text{pkg, if OS} \in \{ \text{FreeBSD} \} \\
\text{tdnf, if OS} \in \{ \text{Photon} \} \\
\end{cases}
$$

It also unpacks `VERSION_ID`s and sets the `APT_KEY_TYPE` for Ubuntu/Debian based distros. I'll use python for representing the `APT_KEY_TYPE` logic; $\LaTeX$ is rather cumbersome and not as pretty as I used to believe.

```python
APT_KEY_TYPE = "keyring"

if (
    (ID in ["ubuntu", "pop", "neon", "zorin", "tuxedo"] and not str(VERSION_ID).startswith("2"))
    or (ID in ["debian", "raspbian"] and VERSION_ID < 11)
    or (ID in ["linuxmint", "parrot", "mendel"] and VERSION_ID < 5)
    or (ID in ["elementary"] and VERSION_ID < 6)
    or (ID in ["kali"] and int(str(VERSION_ID)[0:4]) < 2021)
    or (ID in ["Deepin", "deepin"] and VERSION_ID < 20)
    or (ID in ["galliumos"])
):
    APT_KEY_TYPE = "legacy"
```

Ignoring the wild variety of Linux distros, everything so far seems straightforward.

In [4]:
#| code-fold: show
#| code-summary: Showing my /etc/os-release file's contents

!cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.5 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.5 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [5]:
#| code-fold: show
#| code-summary: The /etc/os-release parsing conditional block

print("\n".join(linux_script_lines.split("\n")[37:304]))

	if [ -f /etc/os-release ]; then
		# /etc/os-release populates a number of shell variables. We care about the following:
		#  - ID: the short name of the OS (e.g. "debian", "freebsd")
		#  - VERSION_ID: the numeric release version for the OS, if any (e.g. "18.04")
		#  - VERSION_CODENAME: the codename of the OS release, if any (e.g. "buster")
		#  - UBUNTU_CODENAME: if it exists, use instead of VERSION_CODENAME
		. /etc/os-release
		case "$ID" in
			ubuntu|pop|neon|zorin|tuxedo)
				OS="ubuntu"
				if [ "${UBUNTU_CODENAME:-}" != "" ]; then
				    VERSION="$UBUNTU_CODENAME"
				else
				    VERSION="$VERSION_CODENAME"
				fi
				PACKAGETYPE="apt"
				# Third-party keyrings became the preferred method of
				# installation in Ubuntu 20.04.
				if expr "$VERSION_ID" : "2.*" >/dev/null; then
					APT_KEY_TYPE="keyring"
				else
					APT_KEY_TYPE="legacy"
				fi
				;;
			debian)
				OS="$ID"
				VERSION="$VERSION_CODENAME"
				PACKAGETYPE="apt"
				# Third-party keyrings became the prefer

If there wasn't a `/etc/os-release` file and the conditional block above was skipped, the `OS` variable will be empty and the shell will enter this block (unless the running context's `env`ironment already set an `OS` variable).

In the block, it checks if the invoking-shell has a command named `uname` (ie checking if the invoking-machine is running Linux/Unix or if it's a Windows machine), and sets the `OS`, `VERSION`, and `PACKAGETYPE` variables based on the string `uname` returns.

Straightforward.

In [6]:
#| code-fold: show
#| code-summary: Showing the output of the uname command
!uname

Linux


In [7]:
#| code-fold: show
#| code-summary: Showing the location of the `uname` program
!type uname

uname is /usr/bin/uname


In [8]:
#| code-fold: show
#| code-summary: The conditional block that determines OS when /etc/os-release doesn't exist

print("\n".join(linux_script_lines.split("\n")[305:336]))

	# If we failed to detect something through os-release, consult
	# uname and try to infer things from that.
	if [ -z "$OS" ]; then
		if type uname >/dev/null 2>&1; then
			case "$(uname)" in
				FreeBSD)
					# FreeBSD before 12.2 doesn't have
					# /etc/os-release, so we wouldn't have found it in
					# the os-release probing above.
					OS="freebsd"
					VERSION="$(freebsd-version | cut -f1 -d.)"
					PACKAGETYPE="pkg"
					;;
				OpenBSD)
					OS="openbsd"
					VERSION="$(uname -r)"
					PACKAGETYPE=""
					;;
				Darwin)
					OS="macos"
					VERSION="$(sw_vers -productVersion | cut -f1-2 -d.)"
					PACKAGETYPE="appstore"
					;;
				Linux)
					OS="other-linux"
					VERSION=""
					PACKAGETYPE=""
					;;
			esac
		fi
	fi


## Internet access check

This block determines whether `curl` or `wget` are available on the invoking-machine, configures the prefix of web-request-making commands (in the `CURL` variable) based on whether the system has `curl` or not, and then uses that web-request tool to check a test URL, pkgs.tailscale.com, to make sure that the machine has access to the internet. If the test fails, the output of `$($CURL "$TEST_URL" 2>&1)` gets saved to the `RC` variable and an error message is displayed.

In [9]:
#| code-fold: show
#| code-summary: The code block that checks if the running machine can access the internet

print("\n".join(linux_script_lines.split("\n")[337:361]))

	# Ideally we want to use curl, but on some installs we
	# only have wget. Detect and use what's available.
	CURL=
	if type curl >/dev/null; then
		CURL="curl -fsSL"
	elif type wget >/dev/null; then
		CURL="wget -q -O-"
	fi
	if [ -z "$CURL" ]; then
		echo "The installer needs either curl or wget to download files."
		echo "Please install either curl or wget to proceed."
		exit 1
	fi

	TEST_URL="https://pkgs.tailscale.com/"
	RC=0
	TEST_OUT=$($CURL "$TEST_URL" 2>&1) || RC=$?
	if [ $RC != 0 ]; then
		echo "The installer cannot reach $TEST_URL"
		echo "Please make sure that your machine has internet access."
		echo "Test output:"
		echo $TEST_OUT
		exit 1
	fi


In [10]:
#| code-fold: show
#| code-summary: Inspecting the TEST_URL site

TEST_URL="https://pkgs.tailscale.com/"
test_url_resp = requests.get(TEST_URL)
test_url_resp.raise_for_status()
display(HTML(test_url_resp.text))

## OS_UNSUPPORTED check
Then, the script checks to see if the detected `OS` is supported, saving that in the variable `OS_UNSUPPORTED`, which is initialized to be empty and reassigned the value `1` only if case-specific conditions are detected. If `OS_UNSUPPORTED == 1`, then the script prints out a message indicating tailscale isn't available for the detected OS-version pair, and a copyable blurb users could email to tailscale support to potentially support future tailscale development.

In [11]:
#| code-fold: show
#| code-summary: The code block that checks if the invoking OS is supported

print("\n".join(linux_script_lines.split("\n")[362:447]))

	# Step 2: having detected an OS we support, is it one of the
	# versions we support?
	OS_UNSUPPORTED=
	case "$OS" in
		ubuntu|debian|raspbian|centos|oracle|rhel|amazon-linux|opensuse|photon)
			# Check with the package server whether a given version is supported.
			URL="https://pkgs.tailscale.com/$TRACK/$OS/$VERSION/installer-supported"
			$CURL "$URL" 2> /dev/null | grep -q OK || OS_UNSUPPORTED=1
			;;
		fedora)
			# All versions supported, no version checking required.
			;;
		arch)
			# Rolling release, no version checking needed.
			;;
		manjaro)
			# Rolling release, no version checking needed.
			;;
		alpine)
			# All versions supported, no version checking needed.
			# TODO: is that true? When was tailscale packaged?
			;;
		void)
			# Rolling release, no version checking needed.
			;;
		gentoo)
			# Rolling release, no version checking needed.
			;;
		freebsd)
			if [ "$VERSION" != "12" ] && \
			   [ "$VERSION" != "13" ] && \
			   [ "$VERSION" != "14" ]
			then
				OS_UNSUP

## Root-privilege detection

The script then checks to see if the running shell can run privileged commands (ie if it is the root user or if it can use the `sudo` or `doas` command).

If the running shell can't run privileged commands, the script outputs a message and exits.

In [12]:
#| code-fold: show
#| code-summary: The code block that checks for ability to run privileged commands

print("\n".join(linux_script_lines.split("\n")[448:468]))


	# Step 3: work out if we can run privileged commands, and if so,
	# how.
	CAN_ROOT=
	SUDO=
	if [ "$(id -u)" = 0 ]; then
		CAN_ROOT=1
		SUDO=""
	elif type sudo >/dev/null; then
		CAN_ROOT=1
		SUDO="sudo"
	elif type doas >/dev/null; then
		CAN_ROOT=1
		SUDO="doas"
	fi
	if [ "$CAN_ROOT" != "1" ]; then
		echo "This installer needs to run commands as root."
		echo "We tried looking for 'sudo' and 'doas', but couldn't find them."
		echo "Either re-run this script as root, or set up sudo/doas."
		exit 1


In [13]:
#| code-fold: show
#| code-summary: If the shell is running as root, the `uid` will be 0

!id -u

1000


In [14]:
#| code-fold: show
#| code-summary: Checks if the shell can access the sudo command

!type sudo

sudo is /usr/bin/sudo


In [15]:
#| code-fold: show
#| code-summary: Checks if the shell can access the doas command

!type doas

/bin/bash: line 1: type: doas: not found


## Running the installation

This block of code actually does the work. I'll dig into the Ubuntu/Debian case, as that's my situation (and Ubuntu/Debian are the most popularly used linux distros by a wide margin).

For machines using the `apt` package manager, this script:
1. sets an env-var `DEBIAN_FRONTEND` to `noninteractive`,
2. ensures `gpg` is usable on the system (well, that `gnupg` is installed),
3. turns on command displaying (via `set -x`),
4. ensures the keyring dir exists, root can read/write to it, and all other groups and users can only read from that dir,
5. downloads the appropriate tailscale public key file and adds the appropriate tailscale package repo to the invoking-machine's local keyring and package-source list,
6. updates package version intel and installs tailscale, and
7. starts `tailscaled`, the tailscale daemon.

In [16]:
#| code-fold: show
#| code-summary: The conditional block that installs tailscale

print("\n".join(linux_script_lines.split("\n")[470:614]))


	# Step 4: run the installation.
	OSVERSION="$OS"
	[ "$VERSION" != "" ] && OSVERSION="$OSVERSION $VERSION"
	echo "Installing Tailscale for $OSVERSION, using method $PACKAGETYPE"
	case "$PACKAGETYPE" in
		apt)
			export DEBIAN_FRONTEND=noninteractive
			if [ "$APT_KEY_TYPE" = "legacy" ] && ! type gpg >/dev/null; then
				$SUDO apt-get update
				$SUDO apt-get install -y gnupg
			fi

			set -x
			$SUDO mkdir -p --mode=0755 /usr/share/keyrings
			case "$APT_KEY_TYPE" in
				legacy)
					$CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.asc" | $SUDO apt-key add -
					$CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.list" | $SUDO tee /etc/apt/sources.list.d/tailscale.list
				;;
				keyring)
					$CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.noarmor.gpg" | $SUDO tee /usr/share/keyrings/tailscale-archive-keyring.gpg >/dev/null
					$CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.tailscale-keyring.list" | $SUDO tee /etc/apt/sources.list.d/tailscale.list
				;;
			esac

In [17]:
#| code-fold: show
#| code-summary: Checking the keys and repos

track = "stable"
os = "ubuntu"
version = "jammy"

# $CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.asc" | $SUDO apt-key add -
legacy_key_url = f"https://pkgs.tailscale.com/{track}/{os}/{version}.asc"
legacy_key_resp = requests.get(legacy_key_url)
legacy_key_resp.raise_for_status()
print("Legacy gpg key")
print(legacy_key_resp.text)

# $CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.list" | $SUDO tee /etc/apt/sources.list.d/tailscale.list
legacy_pkg_repo_url = f"https://pkgs.tailscale.com/{track}/{os}/{version}.list"
legacy_pkg_repo_resp = requests.get(legacy_pkg_repo_url)
legacy_pkg_repo_resp.raise_for_status()
print(f"\n{legacy_pkg_repo_resp.text}")

# $CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.noarmor.gpg" | $SUDO tee /usr/share/keyrings/tailscale-archive-keyring.gpg >/dev/null
keyring_keyfile_url = f"https://pkgs.tailscale.com/{track}/{os}/{version}.noarmor.gpg"
keyring_keyfile_resp = requests.get(keyring_keyfile_url)
keyring_keyfile_resp.raise_for_status()
print(f"\n\nModern keyring gpg key (compressed, binary format)")
print("Omitted due to being unparsable binary")
# print(keyring_keyfile_resp.text)

# $CURL "https://pkgs.tailscale.com/$TRACK/$OS/$VERSION.tailscale-keyring.list" | $SUDO tee /etc/apt/sources.list.d/tailscale.list
keyring_pkg_repo_url = f"https://pkgs.tailscale.com/{track}/{os}/{version}.tailscale-keyring.list"
keyring_pkg_repo_resp = requests.get(keyring_pkg_repo_url)
keyring_pkg_repo_resp.raise_for_status()
print(f"\n{keyring_pkg_repo_resp.text}")

Legacy gpg key
-----BEGIN PGP PUBLIC KEY BLOCK-----

mQINBF5UmbgBEADAA5mxC8EoWEf53RVdlhQJbNnQW7fctUA5yNcGUbGGGTk6XFqO
nlek0Us0FAl5KVBgcS0Bj+VSwKVI/wx91tnAWI36CHeMyPTawdT4FTcS2jZMHbcN
UMqM1mcGs3wEQmKz795lfy2cQdVktc886aAF8hy1GmZDSs2zcGMvq5KCNPuX3DD5
INPumZqRTjwSwlGptUZrJpKWH4KvuGr5PSy/NzC8uSCuhLbFJc1Q6dQGKlQxwh+q
AF4uQ1+bdy92GHiFsCMi7q43hiBg5J9r55M/skboXkNBlS6kFviP+PADHNZe5Vw0
0ERtD/HzYb3cH5YneZuYXvnJq2/XjaN6OwkQXuqQpusB5fhIyLXE5ZqNlwBzX71S
779tIyjShpPXf1HEVxNO8TdVncx/7Zx/FSdwUJm4PMYQmnwBIyKlYWlV2AGgfxFk
mt2VexyS5s4YA1POuyiwW0iH1Ppp9X14KtOfNimBa0yEzgW3CHTEg55MNZup6k2Q
mRGtRjeqM5cjrq/Ix15hISmgbZogPRkhz/tcalK38WWAR4h3N8eIoPasLr9i9OVe
8aqsyXefCrziaiJczA0kCqhoryUUtceMgvaHl+lIPwyW0XWwj+0q45qzjLvKet+V
Q8oKLT1nMr/whgeSJi99f/jE4sWIbHZ0wwR02ZCikKnS05arl3v+hiBKPQARAQAB
tERUYWlsc2NhbGUgSW5jLiAoUGFja2FnZSByZXBvc2l0b3J5IHNpZ25pbmcga2V5
KSA8aW5mb0B0YWlsc2NhbGUuY29tPokCTgQTAQgAOBYhBCWWqZ6qszghiTwKeUWM
qDKVf1hoBQJeVJm4AhsDBQsJCAcCBhUKCQgLAgQWAgMBAh4BAheAAAoJEEWMqDKV
f1hoWHEP/1DYd9WZrodyV5zy1izvj0FXtUReJ

# Conclusion

Looks fine and straightforward. There's nothing sneaky in this script.